<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 4: Distillation and Retrieval Augmented Generation</h2>
<h4>Lecture: Dr. Guokan Shang<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, November 18, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>Novemver 23
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

# Install Requirements

In [1]:
# Install required dependencies
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate vllm
!pip install jedi
# !pip -q install datasets==2.21.0 pandas==2.2.2
# !pip -q install chromadb==0.4.22
# !pip -q install "numpy<2.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/

# Part 1 - Model Distillation

> In this section, you’ll learn the difference between **white-box** and **black-box** distillation, generate **synthetic data** to train a **student model**, and implement **white-box distillation** to specialize a model for a **RAG on Wikipedia** use case.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:</br>
Explain briefly the difference between black-box and white-box distillation? </br> What are the advantages and inconvenients of each approach?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
In white-box distillation, the teacher model is fully accessible: its architecture, parameters and internal representations (such as logits or hidden states) can be used to train the student. Since the student learns from internal signals rather than only from final predictions, the supervision is richer and typically leads to better performance with fewer training examples. However, this approach requires running the full teacher locally, which increases computational cost, and it is impossible when the model is proprietary.
In black-box distillation, the teacher is treated as inaccessible and only its final outputs are observable, often through an API. The student learns to reproduce these outputs without access to the teacher's parameters or intermediate activations. This makes the method compatible with commercial LLMs, but the supervision is weaker and performance usually depends on generating a large amount of synthetic data to compensate for the lack of internal information.
<hr style="border:10px solid green"> </hr>
</font></h4>



<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 2:</br>
What is the main requirement for a teacher/student pair of models to perform white-box distillation?
<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
The main requirement for white-box distillation is full access to the teacher's internal information: its architecture, parameters and intermediate representations. The student must be able to read the teacher's weights and hidden states, so distillation can align or imitate these internal features rather than only its final outputs.
<hr style="border:10px solid green"> </hr>
</font></h4>


## 1.1 - Synthetic Data Generation

We're going to specialize a small 0.5B parameter model to perform RAG by distilling the abilities of a 7B parameter one.

For that we'll be using `Qwen/Qwen2.5-0.5B-Instruct` as student and `Qwen/Qwen2.5-7B-Instruct-AWQ` (quantized version of `Qwen2.5-7B-Instruct`) as teacher.  

In order to perform white-box distillation on generated answers we have two choices.

1. We can perform a forward pass with the teacher, a forward pass with student on the complete sequence, and backprop difference of logprobs using KL Loss.
2. Generation of samples with the teacher, save the logprobs and perform finetuning in a second step.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 3:</br>
What are the computational advantages of 1. vs 2.?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>
In the first approach, the teacher and the student are run in a single pass: we compute the teacher's logits on the fly, immediately compute the student's logits on the same input, and directly backpropagate the KL loss. This avoids storing datasets of generated outputs and teacher log-probabilities, so no intermediate data has to be saved, serialized, or reloaded. As a result, it reduces memory and storage requirements and keeps training fully streamed.
In the second approach, generation and training are separated. The teacher must first produce outputs and log-probabilities that must be saved and later reloaded for training. This two-step workflow increases storage usage, I/O cost, and preprocessing time, although it allows reusing the same dataset across multiple training runs.
<hr style="border:10px solid green"> </hr>
</font></h4>

We're going to generate a bunch of questions related to wikipedia paragraphs.
For that we need to establish a system prompt that will allow for easy extraction.

In [2]:
system_prompt = """
You are a question generator.
The user will provide:

```json
{"title": "the title of an article", "paragraph": "a paragraph from that article"}
```

Your task:

* Generate one clear, self-contained question that can be answered using only the provided paragraph.
* The question must be **specific**, **unambiguous**, and directly tied to the paragraph’s content.
* Return the result with the question as a valid JSON** in the form:

```json
{
  "question": "your question here"
}
```

Example:
User input:

```json
{
"title": "The Moon Landing",
"paragraph": "On July 20, 1969, Neil Armstrong became the first human to set foot on the Moon, followed by Buzz Aldrin."
}
```
Assistant output:

```json
{
  "question": "Who was the first human to set foot on the Moon during the Apollo 11 mission?"
}
```
"""

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 4:</br>
In this system prompt, we don't generate answers, only questions. Explain why it's necessary in the context of white-box distillation.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
In white-box distillation, the goal is to train the student model to imitate the internal behaviour of the teacher (logits, probabilities, hidden states) when producing an answer. If we allowed the teacher to answer as well as generate the questions, then the student would simply learn to imitate the *outputs* directly, drifting toward black-box distillation where only final responses are used. By forcing the prompt to generate only questions, we ensure that the downstream answers will be produced by the models themselves during the training step, so that the distillation loss is computed on their internal predictions. The student is therefore trained using the teacher's internal probabilities over the answer tokens, which is what characterizes white-box distillation.

<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 5:</br>
For synthetic data generation we'll be using vLLM.
vLLM is an optimized llm inference engine that can improve generation speed thanks to hardware specific optimization and computational tricks such as Prefix KV Caching.
(https://docs.vllm.ai/en/latest/features/automatic_prefix_caching.html)</br></br> 1. Explain why prefix caching will be very efficient in our case? </br></br>
2. What sampling `temperature` should we use? Justify.



<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
1.Why is prefix caching very efficient in our case?
In our setting, the teacher will repeatedly generate answers for different synthetic examples, but all of these examples share the same **system prompt**, which remains identical. Prefix KV caching allows vLLM to compute the key - value attention cache for this fixed prefix only once, and then reuse it for every generation request. Since the system prompt is long and identical across all samples, avoiding its repeated forward computation greatly reduces inference time. Most of the computation then focuses only on the variable part (the generated question and answer), making prefix caching highly efficient.

2.What sampling temperature should we use?
For distillation, the goal is not to produce diverse or creative outputs, but to expose the student model to the teacher's typical reasoning patterns and token distribution. A low temperature (close to 0.0 - 0.3) reduces randomness and ensures that the generated answers are stable and representative of the teacher's preferred responses. This avoids noisy or highly variable outputs that would make the student learn unstable distributions. Using a near-deterministic temperature therefore produces consistent teacher behavior, leading to cleaner supervision for white-box distillation.

<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Optimized Generation of Synthetic Questions

Complete the below code with the adequate options (prefix caching and `temperature`)

<hr style="border:10px solid blue"> </hr>
</font></h4>


In [3]:
from vllm import LLM
from vllm import SamplingParams
import os

path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"

llm = LLM(
    model=path_teacher,
    gpu_memory_utilization=0.9,
    max_model_len=5000,
    trust_remote_code=True        # required for Qwen models
)

sampling_params = SamplingParams(
    temperature=0.1,              # low randomness for stable distillation targets
    max_tokens=400
)


INFO 11-23 15:06:04 [utils.py:253] non-default args: {'trust_remote_code': True, 'max_model_len': 5000, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-7B-Instruct-AWQ'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

INFO 11-23 15:06:23 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-23 15:06:23 [model.py:1745] Using max model len 5000
INFO 11-23 15:06:26 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 11-23 15:06:29 [system_utils.py:103] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized


RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Question Generation</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

We're going to use the llm.chat vLLM api to generate our samples.
Make the adequate code to generate the data and save it in a questions.jsonl file.

Entries of jsonl file should look like:
```json
{
  "id_doc": <wikipedia_article_id>,
  "id_paragraph": <paragraph_dataset_id>,
  "question": <generated_question>,
  "title": <title_wikipedia_article>,
  "paragraph": <text_of_paragraph>
}
```

You should:

1. Complete `def extract_question(generated_text: str) -> str:` to extract the generated question.
2. Complete `def conversation_generator` to output a generator directly ingestible by `llm.chat` api
3. Complete the dataloader and for loop to generate samples by batches of 4
4. Save the generated questions in a `questions.jsonl` file

In [9]:
from typing import Iterable, Iterator, List
import json

def extract_question(generated_text: str) -> str:
    """Extract the question from the generated text. If the question is not
    following the right format return None."""
    try:
        j: dict = json.loads(generated_text)   # parse returned JSON
        assert "question" in j
        return j["question"]
    except Exception:  # includes wrong JSON format or missing key
        return None


def conversation_generator(
    entries: Iterable[dict],
    system_prompt: str
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""
    for entry in entries:
        conversation: List[dict] = [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": json.dumps(
                    {"title": entry["title"], "paragraph": entry["paragraph"]}
                )
            }
        ]
        yield conversation

In [12]:
from torch.utils.data import DataLoader
from datasets import load_dataset

ds = load_dataset("EvanD/Lab4_wikiparagraphs")

batch_size = 4
conversations = list(conversation_generator(ds["train"], system_prompt))

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)


In [13]:
from tqdm.auto import tqdm
import jsonlines


with jsonlines.open("questions.jsonl", "w") as writer:
    for i, batch in tqdm(enumerate(dataloader)):
        id_paragraph = i * batch_size
        outputs = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for j, output in enumerate(outputs):
          entry = {}
          q = extract_question(output.outputs[0].text)

          # You must verify that the output actually contains a question and write it
          # To Complete
          if q is not None:
                doc = ds["train"][id_paragraph + j]
                entry = {
                    "id_doc": doc["id_doc"],
                    "id_paragraph": doc["id_paragraph"],
                    "question": q,
                    "title": doc["title"],
                    "paragraph": doc["paragraph"]
                }
                writer.write(entry)

0it [00:00, ?it/s]

NameError: name 'llm' is not defined

## 1.2 - Logprobs Generation

We get to the second part of this distillation where we are interested in distilling the answers logprobs of our teacher model to specialize our 0.5B model to perform Retrieval Augmented Generation (RAG).

First we're going to generate the logprobs with our 7B parameter model.

We'll use the following system prompt:

In [14]:
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: Complete Conversation Generator</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

Based on the previous code you made to generate questions, make a second one to generate answers and saving logprobs. We updated the sampling parameters of vLLM to return the logprobs and the token ids of the 20 most probable tokens.

As this can result in high quantity of data in practice we're going to store generated conversations in a .jsonl file and generated logprobs and token ids to a hdf5 file (see https://docs.h5py.org/en/stable/ for more information on hdf5).

`save_logprobs_hdf5()` is already implemented for you, and allows to save the logprobs to a .h5 hdf5 file, and increments sequences automatically

The structure of the conversation generator `conversation_generator()` function is implemented, you need to complete it.



In [15]:
import h5py, os
import numpy as np

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs  = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs",  data=logprobs,  compression="gzip")



<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 6:</br>
When completing conversation_generator we have several ways of sampling.
What are good paragraph sampling strategies we could use to ensure good performance of downstream model?

<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Good sampling should maximize the informational value of the generated questions. Rather than sampling paragraphs uniformly, we should bias our selection toward paragraphs that are neither too short nor too long (to avoid trivial or noisy supervision), contain richer content such as entities, dates or explanations, and maintain diversity across topics. This ensures the teacher produces varied and meaningful questions, improving the downstream student model.
<hr style="border:10px solid green"> </hr>
</font></h4>

In [16]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]]

        while len(paragraphs) < number_of_paragraphs_in_context:
          # With 50% probability, add a paragraph from the same article
          if random.random() < 0.5:
              same = random.choice(doc_id_to_paragraphs[q_p["id_doc"]])
              if same not in paragraphs:
                  paragraphs.append(same)
                  continue

          # Add a paragraph from a different article
          random_doc = random.choice(list(doc_id_to_paragraphs.keys()))
          if random_doc != q_p["id_doc"]:
              other = random.choice(doc_id_to_paragraphs[random_doc])
              if other not in paragraphs:
                  paragraphs.append(other)
                  continue

        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q_p["question"]}  # FIXED
        ]
        yield conversation


In [17]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Iterable


batch_size = 4

conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt)) # To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

sampling_params = SamplingParams(temperature=0.2, max_tokens=400, logprobs=20)

with jsonlines.open("conversations_rag.jsonl", "w") as writer:
    for batch in tqdm(dataloader):
        out = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for i, b in enumerate(batch):
            text = out[i].outputs[0].text
            b.append({"role": "assistant", "content": text})
            writer.write(b)
        save_logprobs_hdf5("logprobs.h5", [out[i].outputs[0].logprobs for i in range(len(out))])


conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))


0it [00:00, ?it/s]

## 1.3 - KL-Divergence and Distillation

**You should restart the notebook kernel to free the gpu memory from the 7B model that is no longer needed**

Now that we have the generated conversations and their logprobs we can train our 0.5B model to output the same distribution.

The attentive student would have noticed that we have an incomplete representation of the probability distribution over tokens due to only keeping the top-20 logprobs.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 7:</br>
What are the two solutions you can see to approximate full distillation despite only having the top-20 logprobs?

<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Two reasonable approximations can be used when only the top-20 log-probabilities are available instead of the full vocabulary distribution:

(1) Renormalization over the top-k tokens.
We treat the retrieved top-20 probabilities as if they were the only candidates, exponentiate them, and renormalize so they sum to 1. This approximates the teacher distribution on a reduced support and allows a KL loss to be computed.

(2) Add a “remaining mass” bucket.
We compute the total missing probability mass (1 - sum of the top-20 probabilities) and assign it to one synthetic token representing all other unseen tokens. This keeps the distribution closer to the true one and penalizes the student for assigning probability outside the top-20.
<hr style="border:10px solid green"> </hr>
</font></h4>

We supply two functions to help in this implementation:

- `find_subsequence()`that allows to find all the occurences of a token subsequence in a 1-D tensor
- `get_labels()` that allows to expand the top-20 logprobs to the whole vocabulary, implicitly setting prob to zero for other tokens
- `QwenKLDataset`that loads samples from .h5 and .jsonl files, remove problematic inconsistent tokenized examples and outputs samples tokenized for training.

To simplify we will train with a batch size of 1, you can implement gradient accumulation if you wish.

The `PREFIX_ASSISTANT`variable contains the tokens that encode for

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [ ]:
import torch

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""

    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

In [ ]:
from torch.utils.data import Dataset
import jsonlines
import h5py


PREFIX_ASSISTANT = [198, 151644, 77091, 198]

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:

        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()

        return entry

KL-Divergence is a metric often used to quantify the difference of probability mass between two distributions.
It's not mathematically defined as a distance because of its asymetric nature:

$$D_{KL}(P \parallel Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)}$$

In knowledge distillation, we often minimize the **Kullback–Leibler divergence** between the teacher’s output distribution $P_T$ and the student’s output distribution $P_S$.

$$
D_{KL}(P_T \parallel P_S) = \sum_x P_T(x) \log \frac{P_T(x)}{P_S(x)}
$$
$$
D_{KL}(P_S \parallel P_T) = \sum_x P_S(x) \log \frac{P_S(x)}{P_T(x)}
$$


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 8:</br>
What qualitative difference would it make if we minimize $D_{KL}(P_T \parallel P_S)$ instead of $D_{KL}(P_S \parallel P_T)$? How would it affect the convergence of the student distribution?

<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 8: </b><br>
Minimizing DKL(PT | PS) makes the student try to cover all outcomes the teacher considers likely. It penalizes the student mainly when it assigns too little probability to tokens the teacher prefers, which encourages a broader distribution.
Minimizing DKL(PS | PT) instead pushes the student to focus only on the teacher's most probable tokens and ignore alternatives. The student becomes more concentrated and mode-seeking, converging faster but losing diversity compared to the teacher.
<hr style="border:10px solid green"> </hr>
</font></h4>

**You need to ensure you restarted the kernel and have sufficiently free memory (no 7B model running)**

check with `! nvidia-smi`

In [ ]:
! nvidia-smi




This code is very basic, and allows you to test a training over the small number of samples you've generated. Due to limitations of Google Colab we cannot go much further, but this first part should have given you the basics on how to perform white-box distillation.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: Complete The Training Code</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>


- Loss function
- gradient accumulation handling



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np



student_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

dataset = QwenKLDataset(path_h5="logprobs.h5", path_json="questions.jsonl")

num_epochs = 2
grad_accum_steps = 8
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])
num_training_steps = num_epochs * len(loader) // grad_accum_steps
scheduler = get_linear_schedule_with_warmup(optimizer, 0, num_training_steps)
loss_fn = torch.nn.KLDivLoss(reduction="batchmean")

for epoch in range(num_epochs):
    for i, batch in enumerate(loader):
        outputs = student_model(batch["inputs"]["input_ids"])
        logits_student = outputs.logits[:, :-1, :]                     # align with next token
        log_probs_student = torch.log_softmax(logits_student, dim=-1)

        # teacher distribution already stored (normalized or renormalizable)
        teacher_target = batch["teacher_probs"][:, 1:, :]              # same alignment

        batch_loss = loss_fn(log_probs_student, teacher_target)

        batch_loss.backward()
        # To Complete
        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print (f"Epoch {epoch+1}, step {i+1}/{len(loader)}: loss = {batch_loss.item():.4f}")
    print(f"Epoch {epoch+1}: loss = {outputs.loss.item():.4f}")

student_model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")


# Part 2 - Retrieval Augmented Generation (RAG)

In this section, we will discuss the concept of **Retrieval-Augmented Generation (RAG)** — a framework that combines **information retrieval** and **language generation**. RAG enables language models to access **external knowledge sources** at inference time, reducing hallucinations and improving factual accuracy.

We will explore how to:
- Build and index a **Vector database** from a corpus (here: Wikipedia sample).
- Retrieve the most relevant documents given a query using **embedding-based similarity**.
- Integrate retrieval results into the **generation pipeline** to produce context-aware answers.


In [ ]:
# restart the session and run
!pip -q install chromadb==0.4.22
!pip -q install "numpy<2.0" --force-reinstall
!pip -q install datasets==2.21.0 pandas==2.2.2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
import os
import json
import random
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# --- Configuration ---
SEED = 42
NUM_ROWS = 1000
DATA_PATH = "wikipedia_20231101_en_1000.csv"

if os.path.exists(DATA_PATH):
    print(f"✅ Found existing dataset at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    print("⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...")
    random.seed(SEED)

    # Load the Wikipedia dataset
    stream_ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True
    )

    buffered_stream = stream_ds.shuffle(seed=SEED, buffer_size=200_000)

    sampled = []
    for ex in buffered_stream:
        try:
            if int(ex["id"]) % 2 == 0:
                sampled.append(ex)
            if len(sampled) >= NUM_ROWS:
                break
        except:
            continue

    # Create DataFrame
    df = pd.DataFrame(sampled)[["id", "url", "title", "text"]]
    df.to_csv(DATA_PATH, index=False)
    print(f"💾 Dataset saved to {DATA_PATH}")


In [ ]:
#Display Basic Info ---
print("Sampled shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

In [ ]:
# Visualize Random Wikipedia Articles ---

NUM_EXAMPLES = 3  # number of random samples to show
samples = df.sample(NUM_EXAMPLES, random_state=random.randint(0, 10000))

for _, row in samples.iterrows():
    display(HTML(f"""
    <hr style="border:2px solid #ccc">
    <h3><b>Title:</b> {row['title']}</h3>
    <p><b>URL:</b> <a href="{row['url']}" target="_blank">{row['url']}</a></p>
    <p style="text-align: justify;"><b>Text:</b><br>{row['text']}</p>
    """))


### **Document Chunking**

The first step in building a RAG pipeline is **chunking**, where large documents are divided into smaller, semantically coherent pieces.  
Chunking allows the retriever to work on manageable text segments instead of entire documents, improving retrieval precision and reducing computational load.  


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Naive Fixed-Length Chunking  
Split each document into overlapping fixed-length chunks to prepare text for retrieval.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:

def text_splitting(text, chunk_length=300, chunk_overlap=100):
    """
    Splits text into fixed-length chunks with overlap.
    """
    out = []
    stride = chunk_length - chunk_overlap## FILL THE GAP: define the stride as the effective step between chunks
    for i in range(0, len(text), stride):
        chunk = text[i:i + chunk_length] ## FILL THE GAP: extract a substring of size 'chunk_length' starting at 'i'
        out.append(chunk)
    return out

# Apply to all documents
df["naive_chunks"] = df["text"].apply(lambda t: text_splitting(t, chunk_length=300, chunk_overlap=100))

num_chunks = df["naive_chunks"].apply(len)
print(f"Average number of chunks per document: {num_chunks.mean():.2f}")
print(f"Total number of chunks: {num_chunks.sum()}")

example_idx = 0
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]["title"])
print("Original length:", len(df.iloc[example_idx]["text"]))
print("Number of chunks:", len(df.iloc[example_idx]["naive_chunks"]))
print("\nFirst 2 chunks:\n")
for i, c in enumerate(df.iloc[example_idx]["naive_chunks"][:2]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Paragraph-Aware Chunking  
Implement a smarter chunking strategy by using the ('.') as a boundary to split text into sentences or short paragraphs, then regroup them until reaching the desired chunk length.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def text_splitting_paragraph(text, chunk_length=300):
    """
    Splits text by sentences/paragraphs (using '.' as boundary)
    and groups them until reaching the desired chunk length.
    """
    out = []
    paragraph_list = text.split('.') ## FILL THE GAP: split the text into smaller parts using '.' as a separator
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += " " + par ## FILL THE GAP: extend the ongoing chunk with the next segment
            length += len(par) ## FILL THE GAP: increment the total length accordingly
        else:
            if len(current_text) != 0:
                out.append(current_text) ## FILL THE GAP: store the current completed chunk before starting a new one)
            current_text = par ## FILL THE GAP: initialize a new chunk with the current paragraph
            length = len(par) ## FILL THE GAP: reset the chunk length counter
    if len(current_text) > 0:
        out.append(current_text) ## FILL THE GAP: add the last remaining chunk to the list)
    return out


# Apply to all documents
df["paragraph_chunks"] = df["text"].apply(lambda t: text_splitting_paragraph(t, chunk_length=300))

# Compute stats
num_chunks_par = df["paragraph_chunks"].apply(len)
print(f"Average number of paragraph-based chunks per document: {num_chunks_par.mean():.2f}")
print(f"Total number of paragraph-based chunks: {num_chunks_par.sum()}")

# Example comparison
example_idx = 432 #Check out other examples
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]['title'])
print("Original length:", len(df.iloc[example_idx]['text']))
print(f"Character based chunks: {len(df.iloc[example_idx]['naive_chunks'])}")
print(f"Paragraph based chunks: {len(df.iloc[example_idx]['paragraph_chunks'])}")

print("\nParagraph chunk preview:\n")
for i, c in enumerate(df.iloc[example_idx]['paragraph_chunks'][:6]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:  
In the paragraph-aware chunking method above, we simply split Wikipedia text using the '.' delimiter to approximate sentence boundaries.  
Discuss whether this is an effective strategy for creating meaningful chunks in a RAG system.
Propose one or more improved chunking strategies that could better capture document structure — and you may include code snippets to justify or demonstrate your approach.  
<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Splitting on “.” is a weak heuristic for chunking Wikipedia text. It often breaks sentences incorrectly (e.g., “U.S.”, “Dr.”, numbers, URLs) and ignores real document structure. This leads to incomplete chunks and a loss of context, which reduces retrieval quality in RAG.

A better approach is to first detect proper sentence boundaries (e.g., using spaCy or NLTK) and then regroup sentences until a target length is reached. Even more robust is to respect paragraph boundaries and only subdivide large paragraphs when needed. For example:

```python
import nltk
nltk.download("punkt")

def better_chunks(text, max_len=300):
    sents = nltk.sent_tokenize(text)
    out, cur = [], ""
    for s in sents:
        if len(cur) + len(s) < max_len:
            cur += " " + s
        else:
            out.append(cur.strip())
            cur = s
    if cur:
        out.append(cur.strip())
    return out

```
This produces coherent, meaningful units that are more suitable for retrieval.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: Consider a scenario where you want to perform RAG on source code (e.g., Python files, Java classes) instead of natural language text. Would the chunking methods demonstrated above (character-based and sentence/paragraph based with boundaries) work effectively for code? Explain why or why not, and describe how you would approach chunking source code to maintain semantic coherence. What specific characteristics of code structure would you need to consider?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
The chunking methods above are designed for natural language and therefore work poorly on source code. Splitting on character length or on punctuation such as “.” has no semantic meaning in code, and would break elements like function definitions, control blocks, imports, or multi-line expressions. This would produce incomplete chunks that cannot be executed or meaningfully retrieved, and RAG would return fragments lacking context (e.g., half of a function).
For code, chunking must follow syntactic and structural units. Meaningful boundaries include functions, classes, import blocks, docstrings, and comment blocks. A typical strategy is to parse the file using a language parser and group code by logical blocks or by lines belonging to the same node in the syntax tree. Large blocks can then be subdivided while retaining full scopes.
Important characteristics to consider include indentation, brackets {}, scopes, dependencies (e.g., helper functions), and docstrings that describe behavior. Respecting these elements ensures that chunks correspond to complete, understandable units of co
<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
# Saving chunks ---

# Flatten chunks into a new DataFrame
records = []
for _, row in df.iterrows():
    doc_id = row["id"]
    title = row["title"]
    url = row["url"]
    for i, chunk in enumerate(row["paragraph_chunks"]):
        records.append({
            "doc_id": doc_id,
            "title": title,
            "url": url,
            "chunk_id": f"{doc_id}_chunk_{i}",
            "chunk_text": chunk.strip()
        })

# Create the flattened chunks DataFrame
chunks_df = pd.DataFrame(records)
print(f"Total chunks: {len(chunks_df)}")
print(f"Average chunk length: {chunks_df['chunk_text'].apply(len).mean():.2f} characters\n")

# Show example
print("Example rows:")
display(chunks_df.head())

chunks_df.to_csv("wikipedia_chunks.csv", index=False)
print("Chunks saved to 'wikipedia_chunks.csv'")
print(f"Total chunks: {len(chunks_df)}")


## <b>Part II: Embedding</b>

After chunking our documents, the next step is to convert text chunks into vector representations (embeddings). These embeddings capture the semantic meaning of the text in a high-dimensional space, allowing us to measure similarity between chunks and queries mathematically.

We will use **sentence-transformers/all-MiniLM-L6-v2**, a compact and efficient embedding model that produces 384-dimensional embeddings for English text. This model offers a strong balance between performance and computational efficiency, making it well-suited for our RAG pipeline.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill in the <code>embed()</code> function to encode text chunks and generate normalized embeddings using <code>sentence-transformers/all-MiniLM-L6-v2</code>.  
Then, apply it to all documents in <code>chunks_df</code> and store the results.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [1]:
# --- 2.3 Embedding Generation ---
# Load model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"Loaded embedding model: {model_name}")


# --- Define embedding function ---
def embed(text_list, doc_type="document"):
    """
    Encodes a list of texts and returns normalized embeddings.
    """
    encoded = tokenizer(
        [f"search_{doc_type}: {t}" for t in text_list],
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoded)                       # FILL THE GAP: forward pass
        token_embeddings = output.last_hidden_state     # FILL THE GAP: last hidden state
        pooled = token_embeddings.mean(dim=1)           # FILL THE GAP: mean over sequence
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)  # FILL: L2 normalization

    return pooled.cpu()




NameError: name 'AutoTokenizer' is not defined

In [ ]:
# --- Test with one example ---
sample_text = ["Artificial intelligence is transforming the world."]
sample_emb = embed(sample_text)
print(f"Sample embedding shape: {sample_emb.shape}")

# --- Apply to all chunks ---
print(f"\nGenerating embeddings for {len(chunks_df)} chunks...")

emb_list = []
for i in tqdm(range(0, len(chunks_df), 32)):
    batch = chunks_df["chunk_text"].iloc[i:i+32].tolist()
    emb = embed(batch, doc_type="document")
    emb_list.append(emb)

chunk_embeddings = torch.cat(emb_list, dim=0).numpy()
chunks_df["embedding"] = list(chunk_embeddings)

print("\nEmbeddings generated and added to DataFrame.")
print(chunks_df[["chunk_id", "title", "embedding"]].head())

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: In most retrieval systems, embeddings are represented as fixed-size vectors. Can you think of a way to design embeddings that can flexibly adjust their size or level of detail while still preserving meaningful similarity between representations? How could such an approach benefit Retrieval-Augmented Generation (RAG) systems in practice, particularly for improving efficiency or adapting to different computational budgets?

<hr style="border:10px solid red"> </hr>
<i>Hint:</i> You can find the answer in the paper <a href="https://arxiv.org/pdf/2205.13147" target="_blank">Matryoshka Representation Learning</a>.
</font></h4>


Embeddings do not need to be fixed-size vectors. In Matryoshka Representation Learning, a representation is learned so that any prefix of the embedding (e.g., first 64, 128, 256 dimensions) still preserves meaningful similarity. The vector is trained with multiple nested objectives so that smaller “sub-embeddings” approximate the same semantic space as the full one.

This produces a scalable embedding where you can trade off speed vs. accuracy without recomputing features. In a RAG system, this means:

  1) low-dimension embeddings can be used for fast, cheap retrieval on large corpora,

  2) dimension embeddings can refine or rerank only the most promising results.

Thus, a single model can adapt to different budgets (mobile vs. server inference) and do retrieval more efficiently by performing coarse search first, fine search only where needed.

### **Building a Simple Vector Database**

After generating embeddings for all our document chunks, the next step is to **store** them in a structure that allows fast similarity search.  
In this section, we will build a **simple in-memory vector database** using PyTorch tensors.  
Each entry in the database will correspond to a text chunk and its embedding, enabling efficient retrieval based on vector similarity.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill in the code to populate the database by computing embeddings for all text chunks in <code>chunks_df</code> using the <code>embed()</code> function.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def populate_database(chunks_df, batch_size=16):
    """
    Populates a vector database from precomputed chunks_df.
    """
    n_chunks = len(chunks_df)

    sample_emb = embed([chunks_df["chunk_text"].iloc[0]])        # compute sample embedding
    output_dim = sample_emb.shape[1]                              # embedding dimension

    vectorial_database = torch.zeros((n_chunks, output_dim))      # empty tensor to store embeddings
    chunk_list = chunks_df["chunk_text"].tolist()

    print(f"Populating vector database with {n_chunks} chunks...")

    n = 0
    for i in range(0, n_chunks, batch_size):
        batch = chunk_list[i:i + batch_size]                      # select batch of texts
        embeddings = embed(batch)                                 # compute batch embeddings
        vectorial_database[n:n + len(batch)] = embeddings         # store embeddings
        n += len(batch)

    return chunk_list, vectorial_database


In [ ]:
#Build Vector Database
chunk_list, vectorial_database = populate_database(chunks_df)

print("\n✅ Vector database successfully built.")
print(f"Total stored chunks: {len(chunk_list)}")
print(f"Database tensor shape: {tuple(vectorial_database.shape)}")

In [ ]:
#Save vector databse
os.makedirs("vector_db", exist_ok=True)

# Save tensor + chunk list
torch.save(vectorial_database, "vector_db/vectorial_database.pth")

with open("vector_db/chunk_list.json", "w", encoding="utf-8") as f:
    json.dump(chunk_list, f, indent=4, ensure_ascii=False)

print("✅ Saved:")
print(" - vector_db/vectorial_database.pth")
print(" - vector_db/chunk_list.json")

In [ ]:
# Load the database
vectorial_database = torch.load("vector_db/vectorial_database.pth", map_location=device)
vectorial_database.requires_grad_(False)

with open("vector_db/chunk_list.json", "r", encoding="utf-8") as f:
    chunk_list = json.load(f)

print(f"✅ Loaded {len(chunk_list)} chunks.")
print(f"Database shape: {vectorial_database.shape}\n")

# Inspect first few entries
for i, embedding_vector in enumerate(vectorial_database[:5]):
    print(f"Vector {i} → {embedding_vector[:5]}")
    print(f"Text snippet: {chunk_list[i][:300]}\n")


#### **Defining Similarity Metrics for Retrieval**

After populating our vector database with embeddings, the next step in a RAG pipeline is to define a *similarity metric* to measure how close two vectors are in the embedding space.  
Common metrics include **dot product**, **L2 distance**, and **cosine similarity**.  

In most retrieval systems, cosine similarity is preferred because it measures the *angle* between two vectors rather than their magnitude, allowing comparison based purely on semantic direction instead of scale.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill in the code to implement the <code>cosine_similarity()</code> function.  
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def cosine_similarity(query_embeddings, doc_embeddings):
    """
    Computes cosine similarity between query and document embeddings
    using manual normalization.
    """
    query_magnitudes = torch.norm(query_embeddings, dim=1, keepdim=True)      # vector norms
    normalized_queries = query_embeddings / query_magnitudes                  # normalize

    doc_magnitudes = torch.norm(doc_embeddings, dim=1, keepdim=True)          # vector norms
    normalized_docs = doc_embeddings / doc_magnitudes                         # normalize

    similarity_matrix = normalized_queries @ normalized_docs.T                # dot product

    return similarity_matrix



# --- Example test ---
query_embeddings = embed([
    "What is t-SNE?",
    "Who is Laurens van der Maaten?"
], "query")

doc_embeddings = embed([
    "t-SNE is a dimensionality reduction algorithm created by Laurens van der Maaten."
], "document")

with torch.no_grad():
    sim_cos = cosine_similarity(query_embeddings, doc_embeddings)

print("🔍 Example cosine similarity scores:\n", sim_cos)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill in the code to complete the <code>retrieve()</code> function.  
It should encode the input query using <code>embed()</code>, compute similarity with all vectors in <code>vectorial_database</code>, and return the top-<i>k</i> most similar text chunks.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def retrieve(query,
             vectorial_database=vectorial_database,
             chunk_list=chunk_list,
             topk=5,
             verbose=False):
    """
    Retrieves top-k most similar chunks to a query using dot-product similarity.
    """
    with torch.no_grad():
        query_embedding = embed([query])                          # encode query
        query_embedding = query_embedding.to(device)              # move to device
        similarity_scores = query_embedding @ vectorial_database.T # compute similarity
        topk_results = torch.topk(similarity_scores, k=topk, dim=1)

        if verbose:
            for score, idx in zip(topk_results.values[0], topk_results.indices[0]):
                print(f"\nScore: {score:.4f}")
                print(f"Chunk:\n{chunk_list[idx][:500]}\n{'-'*80}")

        retrieved_chunks = [chunk_list[idx] for idx in topk_results.indices[0]]  # select text
        return "\n\n".join(retrieved_chunks)  # concatenate as one string



In [ ]:
# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 4: There are retrieval methods like BM25 that rely on lexical overlap between the query and documents, and others based on dense embeddings that capture semantic similarity beyond exact word matches. Explain how these two approaches differ in how they represent and compare text. Then, discuss how a hybrid retrieval strategy combining both can overcome their respective limitations and improve retrieval performance in RAG systems. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>


In real-world RAG systems, instead of manually storing and comparing vectors, we rely on **vector databases** such as **ChromaDB**, which are optimized for efficient **similarity search**, **indexing**, and **retrieval** at scale.  

These databases provide:
- Fast nearest-neighbor search (e.g., using HNSW graphs)  
- Persistent storage for millions of embeddings  
- Built-in support for different similarity metrics (cosine, L2, inner product)  


In [ ]:
import chromadb
from chromadb.config import Settings

# --- Initialize ChromaDB client ---
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset ensures a clean state
chroma_client.reset()

# --- Create a collection ---
# You can choose the similarity metric: "cosine", "l2", or "ip" (inner product)
collection_name = "wikipedia_chunks"
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine similarity works best for normalized embeddings
)

print(f"✅ Created collection: {collection_name}")


In [ ]:
# Prepare the data
ids = chunks_df["chunk_id"].tolist()
embeddings = chunks_df["embedding"].tolist()
documents = chunks_df["chunk_text"].tolist()

# Ensure all embeddings are plain Python lists
embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]

# Add useful metadata for inspection
metadatas = [
    {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "url": row["url"]
    }
    for _, row in chunks_df.iterrows()
]

# Add to the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

print(f"\n✅ Added {collection.count()} chunks to the collection")
print(f"Collection metadata: {collection.metadata}")


In [ ]:
# Encode the query using our embed() function
query = "When was Luigi Boria born?"
query_embedding = embed([query], doc_type="query")[0].tolist()  # get single vector as list

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3 # number of retrieved results
)

# Display results
print(f"🔎 Query: {query}\n" + "=" * 80)
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"Title: {results['metadatas'][0][i]['title']}")
    print(f"Similarity score: {1 - results['distances'][0][i]:.4f}")  # cosine distance → similarity
    print(f"Text: {results['documents'][0][i][:300]}...")
    print("-" * 80)


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5: </b><br>
ChromaDB and other vector databases often rely on Hierarchical Navigable Small World (HNSW) graphs to perform efficient approximate nearest neighbor search.  
Explain how the HNSW algorithm organizes data to enable fast and accurate retrieval in high-dimensional spaces.  
Why is this structure particularly effective for large-scale embedding collections compared to brute-force search?

<hr style="border:10px solid red"> </hr>
<i>Reference:</i> <a href="https://arxiv.org/pdf/1603.09320" target="_blank">Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs</a>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
BM25 is a lexical method: it represents text using exact word occurrences and compares queries and documents based on how often those same words appear, weighted by their importance in the corpus. It does not understand meaning—two texts using different wording for the same idea will look unrelated. In contrast, dense embeddings represent sentences as continuous vectors capturing semantic similarity, so two passages can be close even if they share few or no words. However, dense models may fail on very specific facts where exact wording matters (names, dates, formulas).
A hybrid strategy combines both signals: BM25 provides strong precision for exact matches, while embeddings recover semantically related content. Together they reduce false negatives (missed paraphrased answers) and false positives (irrelevant semantic matches), leading to more robust and accurate retrieval in RAG systems, especially for factual QA or long-form reasoning.
<hr style="border:10px solid green"> </hr>
</font></h4>


### <b> Two-Stage Retrieval: Dense Retrieval + Reranker</b>

In RAG, the first retrieval step often returns passages that are similar in meaning but not always the most relevant.  
A **reranker** fixes this by re-evaluating the top retrieved chunks using a stronger model that jointly reads the query and each document to assign a more accurate relevance score.


In [ ]:
from sentence_transformers import CrossEncoder

# Load reranker model
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL_NAME)

def retrieve_with_reranker(query, collection, initial_k=5, final_k=3):
    """
    Retrieves and reranks candidate documents for a given query.
    Returns both the initial dense results and reranked results.
    """
    query_embedding = embed([query], doc_type="query")[0].tolist()
    candidates = collection.query(query_embeddings=[query_embedding], n_results=initial_k)

    docs = candidates["documents"][0]
    metas = candidates["metadatas"][0]
    dense_scores = [(1 - s) for s in candidates["distances"][0]]

    pairs = [(query, d) for d in docs]
    ce_scores = reranker.predict(pairs)

    reranked = [
        {
            "title": metas[i].get("title", ""),
            "url": metas[i].get("url", ""),
            "text": docs[i],
            "dense_score": dense_scores[i],
            "rerank_score": float(ce_scores[i]),
        }
        for i in range(len(docs))
    ]
    reranked.sort(key=lambda x: x["rerank_score"], reverse=True)

    return docs, metas, dense_scores, reranked[:final_k]

In [ ]:
# Example usage
query = "Where did Luigi Boria study?" #Try other queries too
docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

# --- Display initial dense retrieval ---
print(f"\nInitial dense retrieval (Top 5):\n" + "=" * 80)
for i, (d, s, m) in enumerate(zip(docs, dense_scores, metas), 1):
    print(f"{i}. {m.get('title', '')}  |  Dense similarity: {s:.4f}")
    print(f"Text: {d[:300].replace('\n', ' ')}")
    print("-" * 80)

# --- Display top reranked results ---
print(f"\nAfter Cross-Encoder Reranking (Top 3):\n" + "=" * 80)
for i, item in enumerate(top_reranked, 1):
    print(f"{i}. {item['title']}")
    print(f"Dense similarity: {item['dense_score']:.4f} | Reranker score: {item['rerank_score']:.4f}")
    print(f"Text: {item['text'][:300].replace('\n', ' ')}")
    print("-" * 80)

![Bi-Encoder vs Cross-Encoder Architecture](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6:  
The figure above compares a Bi-Encoder and a Cross-Encoder architecture.  
Rerankers such as <code>cross-encoder/ms-marco-MiniLM-L-6-v2</code> use the second approach, jointly encoding the query and document through a single transformer.  
Why does this joint encoding typically yield higher retrieval precision, and why is it applied as a second-stage reranker instead of being used directly for large-scale retrieval?
<hr style="border:10px solid red"> </hr>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Joint encoding yields higher precision because the model sees the *query and document together* and can attend across them token-by-token. This allows it to judge whether the document truly answers the query, capturing fine-grained relationships such as exact entity matching, negation, or subtle relevance that a bi-encoder (with separate embeddings) cannot model. It therefore produces more accurate relevance scores.
However, this joint encoding is expensive: each query-document pair must be passed through the transformer, so the cost grows linearly with the number of candidates being compared. It is therefore impractical to apply over millions of documents for first-stage retrieval. Instead, a bi-encoder efficiently retrieves a short candidate list, and the cross-encoder is applied afterward to rerank only the top few hundred or dozens, combining high recall from the first stage with high precision in the second.

<hr style="border:10px solid green"> </hr>
</font></h4>


### **Integrating Retrieved Context into the LLM’s Prompt**

Now that we can retrieve and rerank the most relevant document chunks,  
we integrate them directly into the **language model’s prompt**.  
This step allows the model to **ground its answer on factual context** rather than relying solely on internal knowledge —  
thereby improving accuracy and reducing hallucinations.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
gen_tok = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

def generate(msg, max_new_tokens=128, temperature=0.2):
    messages = [{"role": "user", "content": msg}]
    inputs = gen_tok.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    outputs = gen_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        eos_token_id=gen_tok.eos_token_id,
        pad_token_id=gen_tok.pad_token_id if gen_tok.pad_token_id is not None else gen_tok.eos_token_id
    )

    return gen_tok.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()


# ============================================================
# NO-RAG vs WITH RAG
# ============================================================
query = "When was Luigi Boria born?" #Try other queries

print("ORIGINAL PROMPT\n" + "=" * 60)
print(query)

print("\nANSWER WITHOUT RAG\n" + "=" * 60)
print(generate(query))

docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

context = "\n\n".join(h["text"] for h in top_reranked)[:1600]
rag_prompt = (
    f"Use only the context to answer. If unknown, say you don't know.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\nAnswer:"
)

print("\nAUGMENTED PROMPT\n" + "=" * 60)
print(rag_prompt)

print("\nANSWER WITH RAG\n" + "=" * 60)
print(generate(rag_prompt))


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 7: In the RAG prompt construction step, we simply concatenate the retrieved chunks before the question. Discuss potential issues with this naive approach, such as token limits, redundancy, or irrelevant context dilution. Then, explain how we could select, weight, or summarize the retrieved chunks before injecting them into the prompt to improve generation quality and efficiency. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Naively concatenating all retrieved chunks creates several problems. If many chunks are long, the prompt may exceed the model's token limit, forcing truncation and possibly removing useful context. Different chunks can repeat the same information, wasting space, and irrelevant passages may dilute important signals, making the model more likely to produce generic or incorrect answers.
A better strategy is to process the retrieved context before adding it to the prompt. We can rank or weight chunks by relevance score and include only the top few. Redundant passages can be removed through similarity filtering, and very long chunks can be summarized to preserve key facts in fewer tokens. Another option is to merge chunks into a structured format (e.g., bulleted facts) instead of raw text. These approaches reduce noise and improve both efficiency and answer quality.

<hr style="border:10px solid green"> </hr>
</font></h4>


#### **To go further**

- Experiment with other chunking methods (e.g., semantic or recursive chunking).  
- Explore **LangChain** and **LlamaIndex** for building modular RAG pipelines.  
- Try **hybrid retrieval** combining sparse (BM25) and dense embeddings.  
- Explore more advanced RAG methods such as **RAG-Fusion**, **Self-RAG**, and **Active-RAG**.  
- Experiment with **Matryoshka Representation Learning** for scalable embeddings.  
- Try **fine-tuning rerankers** or **retrievers** for domain-specific data.  
